<a href="https://colab.research.google.com/github/Tiru-Kaggundi/Adv_ML_project/blob/main/RNN_based_sentence_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Paragraph generation using Transformer models:

We will attempt to generate a paragraph of text in Hindi language based on a given prompt. We would attempt to train the model based on available online speech of the Prime Minister of India (Mr Narendra Modi) and examine if the prompt is able to generate a paragraph that matches the style closely. We would also examine if the approach can be extended to some famous authors in Hindi language such as Shri Munshi Premchand.


In [1]:
!pip install torchtext==0.11.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.0 MB 4.1 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:42tcmalloc: large alloc 1147494400 bytes == 0x3a436000 @  0x7f885593a615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB 1.9 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0
ERROR: pip's dependency resolver does n

In [2]:
# On Colab, you'll see ('1.10.2+cu102', '0.11.2')
import torch, torchtext
torch.__version__, torchtext.__version__

('1.10.2+cu102', '0.11.2')

In [26]:
from torchtext.vocab import Vectors
import numpy as np
import random

USE_CUDA = torch.cuda.is_available()

if USE_CUDA:
    DEVICE = torch.device('cuda')
    print("Using cuda.")
else:
    DEVICE = torch.device('cpu')
    print("Using cpu.")

random.seed(30255)
np.random.seed(30255)
torch.manual_seed(30255)
if USE_CUDA:
    torch.cuda.manual_seed(30255)

# Change the following to false when training on
# the full set
DEVELOPING = False    
#DEVELOPING = True

if DEVELOPING:
    print('Small development version')
    BATCH_SIZE = 4
    EMBEDDING_SIZE = 20
    MAX_VOCAB_SIZE = 5000
    TRAIN_DATA_SET = "lm-train-small.txt"
    DEV_DATA_SET = "lm-dev-small.txt"
    TEST_DATA_SET = "lm-test-small.txt"
    BPTT_LENGTH = 8
else:
    print('Full version')
    BATCH_SIZE = 32
    EMBEDDING_SIZE = 650
    MAX_VOCAB_SIZE = 50000
    TRAIN_DATA_SET = "lm-train.txt"
    DEV_DATA_SET = "lm-dev.txt"
    TEST_DATA_SET = "lm-test.txt"
    BPTT_LENGTH = 32



Using cuda.
Full version


In [ ]:
# For uploading data to Colab see, e.g., 
# https://medium.com/@philipplies/transferring-data-from-google-drive-to-google-cloud-storage-using-google-colab-96e088a8c041    
# COLAB = False
COLAB = True
if COLAB:
    from google.colab import drive 
    drive.mount('/content/gdrive')
    PATH = "gdrive/My Drive/AML_Project/datasets_hindi_wiki"
else:
    PATH = "/Users/tiru/Documents/Documents_new/Adv_ML/hindi_wiki_55k"
    
LOG_FILE = "language-model.log"

In [4]:
# This command is to unmount gdrive - need it sometimes
#!fusermount -u gdrive

In [27]:
TEXT = torchtext.legacy.data.Field(lower=True)

train, val, test = torchtext.legacy.datasets.LanguageModelingDataset.splits(path=PATH, 
    train=TRAIN_DATA_SET, validation=DEV_DATA_SET, test=TEST_DATA_SET, text_field=TEXT)

TEXT.build_vocab(train, max_size=MAX_VOCAB_SIZE)
VOCAB_SIZE = len(TEXT.vocab)

print(f'Vocabulary size: {VOCAB_SIZE}')

train_iter, val_iter, test_iter = torchtext.legacy.data.BPTTIterator.splits(
    (train, val, test), batch_size=BATCH_SIZE, device=DEVICE, bptt_len=BPTT_LENGTH, 
    repeat=False)

Vocabulary size: 50002


In [28]:
it = iter(train_iter)
batch=next(it)
print("The first three text/target sequences from the first batch are:\n")
indent = " " * 4
for j in range(3):
    print(indent, f"Text Sequence {j}:", 
          " ".join([TEXT.vocab.itos[i] for i in batch.text[:,j].data]))
    print(indent, f"Target Sequence {j}:",
          " ".join([TEXT.vocab.itos[i] for i in batch.target[:,j].data]))
    print()
 
print(f"Each sequence has BPTT_LENGTH = {BPTT_LENGTH}.\n")
print("Also the sequences continue in the next batch!\n")
batch = next(it)
for j in range(3):
    print(indent, f"Text Sequence {j}:", 
          " ".join([TEXT.vocab.itos[i] for i in batch.text[:,j].data]))
    print(indent, f"Target Sequence {j}:",
          " ".join([TEXT.vocab.itos[i] for i in batch.target[:,j].data]))
    print()

The first three text/target sequences from the first batch are:

     Text Sequence 0: <eos> भारतीय मोर या नीला मोर दक्षिण एशिया के देशी तीतर परिवार का एक बड़ा और चमकीले रंग का पक्षी है, दुनिया के अन्य भागों में यह <unk> के रूप में परिचित
     Target Sequence 0: भारतीय मोर या नीला मोर दक्षिण एशिया के देशी तीतर परिवार का एक बड़ा और चमकीले रंग का पक्षी है, दुनिया के अन्य भागों में यह <unk> के रूप में परिचित है।

     Text Sequence 1: पतला स्तर होता है, परंतु स्त्रियों में <unk> <unk> गर्भाशय गुहा तथा <unk> द्वारा यह बाह्य वातावरण में खुलती है। इस <unk> कला की परतों के द्वारा आशय उदर गुहा में लटके
     Target Sequence 1: स्तर होता है, परंतु स्त्रियों में <unk> <unk> गर्भाशय गुहा तथा <unk> द्वारा यह बाह्य वातावरण में खुलती है। इस <unk> कला की परतों के द्वारा आशय उदर गुहा में लटके रहते

     Text Sequence 2: चार वर्ष के <unk> पर्यवेक्षित चिकित्सा-कार्य से लेकर तीन से छः वर्षों के डॉक्टर की उपाधि तक होती है, जिसमें नैदानिक स्थापन शामिल होती है। अमेरिका में नैदानिक मनोवैज्ञानिक के लगभग आधे
  

In [29]:
import torch
import torch.nn as nn


class RNNLM(nn.Module):
    """ Container module with an linear encoder/embedding, an RNN module, and a linear decoder.
    """

    def __init__(self, rnn_type, vocab_size, embedding_dim, hidden_dim, num_layers, 
                 dropout=0.5):
        ''' Initialize model parameters corresponding to ---
            - embedding layer
            - recurrent neural network layer---one of LSTM, GRU, or RNN---with 
              optionally more than one layer
            - linear layer to map from hidden vector to the vocabulary
            - optionally, dropout layers.  Dropout layers can be placed after 
              the embedding layer or/and after the RNN layer. Dropout within
              an RNN is only applied when there are two or more num_layers.
            - optionally, initialize the model parameters.
            
            The arguments are:
            
            rnn_type: One of 'LSTM', 'GRU', 'RNN_TANH', 'RNN_RELU'
            vocab_size: size of vocabulary
            embedding_dim: size of an embedding vector
            hidden_dim: size of hidden/state vector in RNN
            num_layers: number of layers in RNN
            dropout: dropout probability.
            
        '''
        super(RNNLM, self).__init__()
        
        ## YOUR CODE HERE ##

        # Embedding layer definition: 
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # RNN definition - depnding on input type requested for RNN
        if rnn_type == 'LSTM':
          self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, dropout=dropout)
        elif rnn_type == 'GRU':
          self.rnn = nn.GRU(embedding_dim, hidden_dim, num_layers=num_layers, dropout=dropout)
        elif rnn_type == 'RNN_TANH':
          self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers=num_layers, nonlinearity='tanh', dropout=dropout)
        elif rnn_type == 'RNN_RELU':
          self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers=num_layers, nonlinearity='relu', dropout=dropout)
        else:
          print("Enter one of the four inputs : LSTM/GRU/RNN_TANH/RNN_RELU")  
        
        self.fc = nn.Linear(hidden_dim, vocab_size) 


    def forward(self, input, hidden0):
        ''' 
        Run forward propagation for a given minibatch of inputs using
        hidden0 as the initial hidden state.

        In LSTMs hidden0 = (h_0, c_0). 

        The output of the RNN includes the hidden vector hiddenn = (h_n, c_n).
        Return this as well so that it can be used to initialize the next
        batch.
        
        Unlike previous homework sets do not apply softmax or logsoftmax here, since we'll use
        the more efficient CrossEntropyLoss.  See 
        https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html.
        '''

        embeds = self.embedding(input)
        output_n, hidden_n = self.rnn(embeds, hidden0)
        output_n = self.fc(output_n)

        return output_n, hidden_n

In [30]:
def evaluate(model, data):
    '''
    Evaluate the model on the given data.
    '''

    model.eval()
    it = iter(data)
    total_count = 0. # Number of target words seen
    total_loss = 0. # Loss over all target words
    with torch.no_grad():
        # No gradients need to be maintained during evaluation
        # There are no hidden tensors for the first batch, and so will default to zeros.
        hidden = None 
        for i, batch in enumerate(it):
            ''' Do the following:
                - Extract the text and target from the batch, and if using CUDA (essentially, using GPUs), place 
                  the tensors on cuda, using a commands such as "text = text.cuda()".  More details are at
                  https://pytorch.org/docs/stable/notes/cuda.html.
                - Pass the hidden state vector from output of previous batch as the initial hidden vector for
                  the current batch. 
                - Call forward propagation to get output and final hidden state vector.
                - Compute the cross entropy loss
                - The loss_fn computes the average loss per target word in the batch.  Count the number of target
                  words in the batch (it is usually the same, except for the last batch), and use it to track the 
                  total count (of target words) and total loss see so far over all batches.
            '''
            text, target = batch.text, batch.target
            if USE_CUDA:
                text, target = text.cuda(), target.cuda()
            output, hidden = model(text, hidden)
            loss = loss_fn(output.view(-1, output.size(-1)), target.view(-1))
                  
            total_count += np.multiply(*text.size())
            total_loss += loss.item()*np.multiply(*text.size())
                
    loss = total_loss / total_count
    model.train()
    return loss

In [31]:
GRAD_CLIP = 1.
NUM_EPOCHS = 3
CUDA_LAUNCH_BLOCKING = "1"

def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if h is None:
        return None
    elif isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)


''' Do the following:
    - Extract the text and target from the batch, and if using CUDA (essentially, using GPUs), place 
      the tensors on cuda, using a commands such as "text = text.cuda()".  More details are at
      https://pytorch.org/docs/stable/tensors.html#torch.Tensor.cuda -DONE
    - Pass the hidden state vector from output of previous batch as the initial hidden vector for
      the current batch. But detach each tensor in the hidden state vector using tensor.detach() or
      the provided repackage_hidden(). See
      https://pytorch.org/docs/master/generated/torch.Tensor.detach_.html#torch-tensor-detach
    - Zero out the model gradients to reset backpropagation for current batch
    - Call forward propagation to get output and final hidden state vector.
    - Compute the cross entropy loss
    - Run back propagation to set the gradients for each model parameter.
    - Clip the gradients that may have exploded. See Sec 5.2.4 in the Goldberg textbook, and
      https://pytorch.org/docs/master/generated/torch.nn.utils.clip_grad_norm_.html#torch-nn-utils-clip-grad-norm
    - Run a step of gradient descent. 
    - Print the batch loss after every few iterations. (Say every 100 when developing, every 1000 otherwise.)
    - Evaluate your model on the validation set after every, say, 10000 iterations and save it to val_losses. If
      your model has the lowest validation loss so far, copy it to best_model. For that it is recommended that
      copy the state_dict rather than use deepcopy, since the latter doesn't work on Colab.  See discussion at 
      https://discuss.pytorch.org/t/deep-copying-pytorch-modules/13514. This is Early Stopping and is described
      in Sec 2.3.1 of Lecture notes by Cho: 
      https://github.com/nyu-dl/NLP_DL_Lecture_Note/blob/master/lecture_note.pdf
'''


model = RNNLM("LSTM", VOCAB_SIZE, EMBEDDING_SIZE, EMBEDDING_SIZE, 2, dropout=0.5)
print("model:", model)
print("use_cuda_status: ", USE_CUDA)
if USE_CUDA:
  model = model.cuda()

loss_fn = nn.CrossEntropyLoss() ## Used instead of NLLLoss.
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
val_losses = []
best_model = RNNLM("LSTM", VOCAB_SIZE, EMBEDDING_SIZE, EMBEDDING_SIZE, 2, dropout=0.5)


for epoch in range(NUM_EPOCHS):
    model.train()
    log_interval = 1000
    it = iter(train_iter)
    # There are no hidden tensors for the first batch, and so will default to zeros.
    hidden = None

    for i, batch in enumerate(it):
      model.zero_grad()
      text, target = batch.text, batch.target
      if USE_CUDA:
        text, target = text.cuda(), target.cuda()
      output_n, hidden_n = model(text, hidden)
      hidden = repackage_hidden(hidden_n)
      loss = loss_fn(output_n.view(-1, output_n.size(-1)), target.view(-1))

      loss.backward()
      optimizer.step()
      nn.utils.clip_grad_norm_(model.parameters(), max_norm=GRAD_CLIP, norm_type=2.0, error_if_nonfinite=False)

      if i % log_interval == 0 and i > 0:
        print(f'At iteration {i} the loss is {loss:.3f}.')

      if i % 2000 == 0 and i > 0:
        val_loss = evaluate(model, val_iter)
        print(f'Iteration {i}, the validation loss is {val_loss:.3f}.')
        val_losses.append(val_loss)
        if val_loss <= min(val_losses):
          torch.save(model.state_dict(), "gdrive/My Drive/AML_Project/best_model.pt")

best_model.load_state_dict(torch.load("gdrive/My Drive/AML_Project/best_model.pt")) 

model: RNNLM(
  (embedding): Embedding(50002, 650)
  (rnn): LSTM(650, 650, num_layers=2, dropout=0.5)
  (fc): Linear(in_features=650, out_features=50002, bias=True)
)
use_cuda_status:  True
At iteration 1000 the loss is 6.524.
At iteration 2000 the loss is 6.092.
Iteration 2000, the validation loss is 6.088.
At iteration 3000 the loss is 5.893.
At iteration 4000 the loss is 5.484.
Iteration 4000, the validation loss is 5.728.
At iteration 5000 the loss is 5.779.
At iteration 1000 the loss is 5.283.
At iteration 2000 the loss is 5.282.
Iteration 2000, the validation loss is 5.522.
At iteration 3000 the loss is 5.198.
At iteration 4000 the loss is 4.797.
Iteration 4000, the validation loss is 5.392.
At iteration 5000 the loss is 5.117.
At iteration 1000 the loss is 4.678.
At iteration 2000 the loss is 4.820.
Iteration 2000, the validation loss is 5.478.
At iteration 3000 the loss is 4.809.
At iteration 4000 the loss is 4.391.
Iteration 4000, the validation loss is 5.351.
At iteration 500

<All keys matched successfully>

In [32]:
'''
Evaluate the loss of best_model on the validation set and compute its perplexity.
'''
if USE_CUDA:
  best_model = best_model.cuda()
val_loss = evaluate(best_model, val_iter)
print("perplexity: ", np.exp(val_loss))

perplexity:  210.81116793857507


In [33]:
'''
Evaluate the loss of best_model on the test set and compute its perplexity.
'''
test_loss = evaluate(best_model, test_iter)
print("perplexity: ", np.exp(test_loss))

perplexity:  205.139284079365


In [35]:
# Sentence generation in Hindi
# Reference: https://www.analyticsvidhya.com/blog/2020/08/build-a-natural-language-generation-nlg-system-using-pytorch/

import torch.nn.functional as F
# predict next token
def predict(best_model, tkn, h=None):
         
  # tensor inputs
  x = np.array([[TEXT.vocab.stoi[tkn]]])
  inputs = torch.from_numpy(x)
  
  # push to GPU
  inputs = inputs.cuda()

  # detach hidden state from history
  h = repackage_hidden(h)

  # get the output of the model
  out, h = best_model(inputs, h)

  # get the token probabilities
  p = F.softmax(out.view(-1, out.size(-1)) , dim=1)

  # get indices of top 3 or 5 values
  top_n_idx = p.argsort().reshape(-1)[-3:]
  #top_n_idx = p.argsort()[-1][-3:]

  # randomly select one of the three indices
  sampled_token_index = top_n_idx[random.sample([0,1,2],1)[0]]
  #print(sampled_token_index)


  # return the encoded value of the predicted char and the hidden state
  return TEXT.vocab.itos[sampled_token_index], h


# function to generate text
def sample(best_model, size, prime='it is'):
        
    # push to GPU
    #best_model.cuda()
    
    best_model.eval()

    # batch size is 1
    h = None

    toks = prime.split()

    # predict next token
    for t in prime.split():
      token, h = predict(best_model, t, h)
    
    toks.append(token)

    # predict subsequent tokens
    for i in range(size-1):
        token, h = predict(best_model, toks[-1], h)
        toks.append(token)

    return ' '.join(toks)

words_list = ["सिनेमा", "प्राचीन", "मलयालम", "शरीर", "भौतिक", "प्रयोग", "एक"]
for i in words_list:
  print(i," :", sample(best_model, 50, prime=i))

सिनेमा  : सिनेमा <unk> <unk> <unk> के नाम के रूप में, <unk> <unk> <unk> और <unk> में <unk> के <unk> <unk> के <unk> के साथ ही साथ <unk> <unk> और अनुनासिकता का उपयोग किया गया था, जिसे उन्होंने एक <unk> में <unk> के रूप में, <unk> की रचना के लिए <unk> <unk> की
प्राचीन  : प्राचीन <unk> <unk> <unk> और <unk> <eos> <eos> <eos> <unk> <unk> <unk> <eos> <eos> <unk> improve <unk> · difference of china of conservation · south <unk> <unk> <eos> <unk> <unk> · <unk> <unk> <eos> <unk> · difference <eos> m: pso/psi <eos> <unk> <unk> · more... · republic · south ossetia <unk>
मलयालम  : मलयालम <unk> <unk> <unk> <unk> <eos> <eos> अम्मा के एक <unk> में से कुछ <unk> का उपयोग करके यह एक महत्वपूर्ण कवयित्री था। इस प्रकार की <unk> के कारण <unk> <unk> के नाम के रूप में, यह एक महत्वपूर्ण कवयित्री है, जिसे उन्होंने <unk> कहा है। इस बात की पुष्टि करते
शरीर  : शरीर का उपयोग करके <unk> के <unk> <unk> के साथ एक साथ <unk> <unk> के रूप में, यह एक विशेष प्रकार की <unk> <unk> <unk> <unk> <unk> और <unk> में <unk> और <unk>

In [36]:
sen1 = ("तस्मानियाई डैविल एक मांसाहारी धानीप्राणी है जो अब केवल ऑस्ट्रेलिया के द्वीप राज्य तस्मानिया के जंगलों में ही पाया जाता है।" 
        "इसका आकार एक"
        "छोटे कुत्ते के बराबर होता है।"
         "1936 में थायलेसीन​ के विलुप्त होने के बाद यह दुनिया का सबसे बड़ा मांसाहारी धानीप्राणी बन गया।")

sen2 = ("तस्मानियाई डैविल एक मांसाहारी धानीप्राणी है जो अब केवल ऑस्ट्रेलिया के द्वीप राज्य तस्मानिया के जंगलों में ही पाया जाता है।" 
        "इसका आकार" 
        "समुद्र के बराबर होता है।"
         "1936 में थायलेसीन​ के विलुप्त होने के बाद यह दुनिया का सबसे बड़ा मांसाहारी धानीप्राणी बन गया।")

sen3 = ("तस्मानियाई डैविल एक मांसाहारी धानीप्राणी है जो अब केवल ऑस्ट्रेलिया के द्वीप राज्य तस्मानिया के जंगलों में ही पाया जाता है।" 
        "इसका आकार" 
        "एक पेंसिल के बराबर होता है।"
         "1936 में थायलेसीन​ के विलुप्त होने के बाद यह दुनिया का सबसे बड़ा मांसाहारी धानीप्राणी बन गया।")

sen4 = ("डैविल एक मांसाहारी प्रधानमंत्री है जो अब केवल ऑस्ट्रेलिया के द्वीप राज्य तस्मानिया के जंगलों में ही पाया जाता है।" 
        "इसका आकार क्यूबा बराबर होता है।"
         "प्लास्टिक के विलुप्त होने के बाद यह दुनिया का सबसे बड़ा मांसाहारी धानीप्राणी बन गया।")

sen5 = sen1.split()
random.shuffle(sen5)
sen5 = " ".join(sen5)

sen6 = " ".join(['डैविल एक मांसाहारी धानीप्राणी है']*8)

sen_list = [sen1, sen2, sen3, sen4, sen5, sen6]

for sen in sen_list:

    print(sen)
    with open(PATH + "temp_sentence.txt", 'w') as text_file:
        print(sen, file = text_file)

    temp_ds = torchtext.legacy.datasets.LanguageModelingDataset(path=PATH + 'temp_sentence.txt', text_field=TEXT)


    sen_iter = torchtext.legacy.data.BPTTIterator(temp_ds, batch_size=BATCH_SIZE, device=DEVICE, 
                                                  bptt_len=BPTT_LENGTH, repeat=False)
        
    sen_loss = evaluate(best_model, sen_iter)
    print("perplexity: ", np.exp(sen_loss))
    print()

तस्मानियाई डैविल एक मांसाहारी धानीप्राणी है जो अब केवल ऑस्ट्रेलिया के द्वीप राज्य तस्मानिया के जंगलों में ही पाया जाता है।इसका आकार एकछोटे कुत्ते के बराबर होता है।1936 में थायलेसीन​ के विलुप्त होने के बाद यह दुनिया का सबसे बड़ा मांसाहारी धानीप्राणी बन गया।
perplexity:  16106.210474284047

तस्मानियाई डैविल एक मांसाहारी धानीप्राणी है जो अब केवल ऑस्ट्रेलिया के द्वीप राज्य तस्मानिया के जंगलों में ही पाया जाता है।इसका आकारसमुद्र के बराबर होता है।1936 में थायलेसीन​ के विलुप्त होने के बाद यह दुनिया का सबसे बड़ा मांसाहारी धानीप्राणी बन गया।
perplexity:  15410.408300338655

तस्मानियाई डैविल एक मांसाहारी धानीप्राणी है जो अब केवल ऑस्ट्रेलिया के द्वीप राज्य तस्मानिया के जंगलों में ही पाया जाता है।इसका आकारएक पेंसिल के बराबर होता है।1936 में थायलेसीन​ के विलुप्त होने के बाद यह दुनिया का सबसे बड़ा मांसाहारी धानीप्राणी बन गया।
perplexity:  10440.019782252652

डैविल एक मांसाहारी प्रधानमंत्री है जो अब केवल ऑस्ट्रेलिया के द्वीप राज्य तस्मानिया के जंगलों में ही पाया जाता है।इसका आकार क्यूबा बराबर होता है

In [37]:
#(i) Substitute LSTM with a GRU

model_GRU = RNNLM("GRU", VOCAB_SIZE, EMBEDDING_SIZE, EMBEDDING_SIZE, 2, dropout=0.5)
print("model_GRU:", model_GRU)
print("use_cuda_status: ", USE_CUDA)
if USE_CUDA:
  model_GRU = model_GRU.cuda()

loss_fn = nn.CrossEntropyLoss() ## Used instead of NLLLoss.
learning_rate = 0.001
optimizer = torch.optim.Adam(model_GRU.parameters(), lr=learning_rate)
val_losses = []
best_model_GRU = RNNLM("GRU", VOCAB_SIZE, EMBEDDING_SIZE, EMBEDDING_SIZE, 2, dropout=0.5)


for epoch in range(NUM_EPOCHS):
    model_GRU.train()
    log_interval = 1000
    it = iter(train_iter)
    # There are no hidden tensors for the first batch, and so will default to zeros.
    hidden = None

    for i, batch in enumerate(it):
      model_GRU.zero_grad()
      text, target = batch.text, batch.target
      if USE_CUDA:
        text, target = text.cuda(), target.cuda()
      output_n, hidden_n = model_GRU(text, hidden)
      hidden = repackage_hidden(hidden_n)
      loss = loss_fn(output_n.view(-1, output_n.size(-1)), target.view(-1))

      loss.backward()
      optimizer.step()
      nn.utils.clip_grad_norm_(model_GRU.parameters(), max_norm=GRAD_CLIP, norm_type=2.0, error_if_nonfinite=False)

      if i % log_interval == 0 and i > 0:
        print(f'At iteration {i} the loss is {loss:.3f}.')

      if i % 2000 == 0 and i > 0:
        val_loss = evaluate(model_GRU, val_iter)
        print(f'Iteration {i}, the validation loss is {val_loss:.3f}.')
        val_losses.append(val_loss)
        if val_loss <= min(val_losses):
          torch.save(model_GRU.state_dict(), "gdrive/My Drive/AML_Project/best_model_GRU.pt")

best_model_GRU.load_state_dict(torch.load("gdrive/My Drive/AML_Project/best_model_GRU.pt")) 

model_GRU: RNNLM(
  (embedding): Embedding(50002, 650)
  (rnn): GRU(650, 650, num_layers=2, dropout=0.5)
  (fc): Linear(in_features=650, out_features=50002, bias=True)
)
use_cuda_status:  True
At iteration 1000 the loss is 7.806.
At iteration 2000 the loss is 7.722.
Iteration 2000, the validation loss is 7.714.
At iteration 3000 the loss is 7.492.
At iteration 4000 the loss is 7.517.
Iteration 4000, the validation loss is 7.644.
At iteration 5000 the loss is 7.649.
At iteration 1000 the loss is 6.857.
At iteration 2000 the loss is 5.979.
Iteration 2000, the validation loss is 6.124.
At iteration 3000 the loss is 5.789.
At iteration 4000 the loss is 5.285.
Iteration 4000, the validation loss is 5.674.
At iteration 5000 the loss is 5.680.
At iteration 1000 the loss is 5.335.
At iteration 2000 the loss is 5.324.
Iteration 2000, the validation loss is 5.510.
At iteration 3000 the loss is 5.299.
At iteration 4000 the loss is 4.803.
Iteration 4000, the validation loss is 5.462.
At iteration 

<All keys matched successfully>

In [41]:
# Perplexity score on validation and test datasets
'''
Evaluate the loss of best_model on the validation set and compute its perplexity.
'''
if USE_CUDA:
  best_model_GRU = best_model_GRU.cuda()
val_loss = evaluate(best_model_GRU, val_iter)
print("perplexity on validation set: ", np.exp(val_loss))

'''
Evaluate the loss of best_model on the test set and compute its perplexity.
'''
test_loss = evaluate(best_model_GRU, test_iter)
print("perplexity on test set: ", np.exp(test_loss))

perplexity on validation set:  235.55341061618014
perplexity on test set:  234.98294424452448


In [42]:
# Sentence generation in Hindi using GRU model

import torch.nn.functional as F
# predict next token
def predict(best_model_GRU, tkn, h=None):
         
  # tensor inputs
  x = np.array([[TEXT.vocab.stoi[tkn]]])
  inputs = torch.from_numpy(x)
  
  # push to GPU
  inputs = inputs.cuda()

  # detach hidden state from history
  h = repackage_hidden(h)

  # get the output of the model
  out, h = best_model(inputs, h)

  # get the token probabilities
  p = F.softmax(out.view(-1, out.size(-1)) , dim=1)

  # get indices of top 3 or 5 values
  top_n_idx = p.argsort().reshape(-1)[-3:]
  #top_n_idx = p.argsort()[-1][-3:]

  # randomly select one of the three indices
  sampled_token_index = top_n_idx[random.sample([0,1,2],1)[0]]
  #print(sampled_token_index)

  # return the encoded value of the predicted char and the hidden state
  return TEXT.vocab.itos[sampled_token_index], h


# function to generate text
def sample(model, size, prime='it is'):
    
    model.eval()

    # batch size is 1
    h = None

    toks = prime.split()

    # predict next token
    for t in prime.split():
      token, h = predict(best_model, t, h)
    
    toks.append(token)

    # predict subsequent tokens
    for i in range(size-1):
        token, h = predict(best_model, toks[-1], h)
        toks.append(token)

    return ' '.join(toks)

words_list = ["सिनेमा", "प्राचीन", "मलयालम", "शरीर", "भौतिक", "प्रयोग", "एक"]
for i in words_list:
  print(i," :", sample(best_model_GRU, 50, prime=i))

सिनेमा  : सिनेमा में एक <unk> के लिए एक महत्वपूर्ण भूमिका निभाते थे। इस परियोजना ने इस परियोजना को बढ़ावा देने के लिए इस बात का समर्थन किया है और इस बात को देखते हैं और यह एक बार जब <unk> की <unk> के साथ ही साथ साथ एक साथ दिया जा सकता
प्राचीन  : प्राचीन <unk> के रूप मे भी किया गया है। <eos> इस बात पर सहमत है और यह भी देखें: <eos> <unk> के <unk> <unk> <unk> के <unk> में स्थित एक एक प्रमुख <unk> के साथ <unk> <unk> के साथ एक <unk> और <unk> के रूप में, एक नया रूप <unk> <unk>
मलयालम  : मलयालम में एक <unk> <unk> का प्रयोग <unk> में <unk> और <unk> <unk> की रचना करने वाले एक नए स्थान के साथ <unk> की रचना के रूप में <unk> <unk> के लिए एक विशेष प्रकार के संयुक्‍ताक्षर के साथ <unk> <unk> के रूप में वर्णित किया जा रहा है। यह संयुक्‍त
शरीर  : शरीर के लिए किया जा सकता है और <unk> में <unk> <unk> और अन्य सभी प्रकार का उपयोग किया गया है और वे भी <unk> की ओर ले जाती हैं और वे एक दूसरे के साथ एक साथ काम करने की अनुमति देते हैं, जैसे :– कहाँ, और रचनावाद, जैसे <unk>
भौतिक  : भौतिक <unk> के <unk> <unk> <un

In [43]:
# Perplexity scoring for the 5 setences using GRU model

sen1 = ("तस्मानियाई डैविल एक मांसाहारी धानीप्राणी है जो अब केवल ऑस्ट्रेलिया के द्वीप राज्य तस्मानिया के जंगलों में ही पाया जाता है।" 
        "इसका आकार एक"
        "छोटे कुत्ते के बराबर होता है।"
         "1936 में थायलेसीन​ के विलुप्त होने के बाद यह दुनिया का सबसे बड़ा मांसाहारी धानीप्राणी बन गया।")

sen2 = ("तस्मानियाई डैविल एक मांसाहारी धानीप्राणी है जो अब केवल ऑस्ट्रेलिया के द्वीप राज्य तस्मानिया के जंगलों में ही पाया जाता है।" 
        "इसका आकार" 
        "समुद्र के बराबर होता है।"
         "1936 में थायलेसीन​ के विलुप्त होने के बाद यह दुनिया का सबसे बड़ा मांसाहारी धानीप्राणी बन गया।")

sen3 = ("तस्मानियाई डैविल एक मांसाहारी धानीप्राणी है जो अब केवल ऑस्ट्रेलिया के द्वीप राज्य तस्मानिया के जंगलों में ही पाया जाता है।" 
        "इसका आकार" 
        "एक पेंसिल के बराबर होता है।"
         "1936 में थायलेसीन​ के विलुप्त होने के बाद यह दुनिया का सबसे बड़ा मांसाहारी धानीप्राणी बन गया।")

sen4 = ("डैविल एक मांसाहारी प्रधानमंत्री है जो अब केवल ऑस्ट्रेलिया के द्वीप राज्य तस्मानिया के जंगलों में ही पाया जाता है।" 
        "इसका आकार क्यूबा बराबर होता है।"
         "प्लास्टिक के विलुप्त होने के बाद यह दुनिया का सबसे बड़ा मांसाहारी धानीप्राणी बन गया।")

sen5 = sen1.split()
random.shuffle(sen5)
sen5 = " ".join(sen5)

sen6 = " ".join(['डैविल एक मांसाहारी धानीप्राणी है']*8)

sen_list = [sen1, sen2, sen3, sen4, sen5, sen6]

for sen in sen_list:

    print(sen)
    with open(PATH + "temp_sentence.txt", 'w') as text_file:
        print(sen, file = text_file)

    temp_ds = torchtext.legacy.datasets.LanguageModelingDataset(path=PATH + 'temp_sentence.txt', text_field=TEXT)


    sen_iter = torchtext.legacy.data.BPTTIterator(temp_ds, batch_size=BATCH_SIZE, device=DEVICE, 
                                                  bptt_len=BPTT_LENGTH, repeat=False)
        
    sen_loss = evaluate(best_model_GRU, sen_iter)
    print("perplexity: ", np.exp(sen_loss))
    print()

तस्मानियाई डैविल एक मांसाहारी धानीप्राणी है जो अब केवल ऑस्ट्रेलिया के द्वीप राज्य तस्मानिया के जंगलों में ही पाया जाता है।इसका आकार एकछोटे कुत्ते के बराबर होता है।1936 में थायलेसीन​ के विलुप्त होने के बाद यह दुनिया का सबसे बड़ा मांसाहारी धानीप्राणी बन गया।
perplexity:  2255.8927662471897

तस्मानियाई डैविल एक मांसाहारी धानीप्राणी है जो अब केवल ऑस्ट्रेलिया के द्वीप राज्य तस्मानिया के जंगलों में ही पाया जाता है।इसका आकारसमुद्र के बराबर होता है।1936 में थायलेसीन​ के विलुप्त होने के बाद यह दुनिया का सबसे बड़ा मांसाहारी धानीप्राणी बन गया।
perplexity:  1774.3572834034035

तस्मानियाई डैविल एक मांसाहारी धानीप्राणी है जो अब केवल ऑस्ट्रेलिया के द्वीप राज्य तस्मानिया के जंगलों में ही पाया जाता है।इसका आकारएक पेंसिल के बराबर होता है।1936 में थायलेसीन​ के विलुप्त होने के बाद यह दुनिया का सबसे बड़ा मांसाहारी धानीप्राणी बन गया।
perplexity:  1664.1740563314036

डैविल एक मांसाहारी प्रधानमंत्री है जो अब केवल ऑस्ट्रेलिया के द्वीप राज्य तस्मानिया के जंगलों में ही पाया जाता है।इसका आकार क्यूबा बराबर होता है